### A rajouter/faire : 
- Données grèves métérologique et vacance

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


import numpy as np
import pandas as pd
import datetime as dt
from sqlalchemy import create_engine
import os 

# CHARGEMENT DES DONNEES 

In [3]:
dialect = 'mysql+pymysql'
user = 'root'
password = os.getenv('mdp_mySQL')
host = 'localhost'
port = 3306
database_idf = 'SNCF_TGV_TER'

engine_url = f"{dialect}://{user}:{password}@{host}:{port}/{database_idf}"
engine = create_engine(engine_url)

In [4]:
df_disruption = pd.read_sql('disruptions_tgv_ter',engine)
df_disruption

,id_disruption,vehicle_id,train_type,id_stop,name_stop,lon,lat,disruption_start,disruption_end,arrival_delay,departure_delay,cause_delay,data_date
0,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,TGV,stop_point:SNCF:87391003:LongDistanceTrain,Paris - Montparnasse - Hall 1 & 2,2.320510,48.8412,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,Défaut d'alimentation électrique,2024-03-18
1,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,TGV,stop_point:SNCF:87393702:LongDistanceTrain,Massy TGV,2.261250,48.7258,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,Défaut d'alimentation électrique,2024-03-18
2,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,TGV,stop_point:SNCF:87581009:LongDistanceTrain,Bordeaux Saint-Jean,-0.556697,44.8259,2024-03-18 18:46:00,2024-03-18 21:49:00,0.0,0.0,Défaut d'alimentation électrique,2024-03-18
3,dc1eeb61-3ef0-4bfe-b63b-8d52ec7577c5,SNCF:2024-03-18:12254:1187:LongDistanceTrain,TGV,stop_point:SNCF:87391003:LongDistanceTrain,Paris - Montparnasse - Hall 1 & 2,2.320510,48.8412,2024-03-18 10:46:00,2024-03-18 12:58:00,5.0,5.0,Régulation du trafic,2024-03-18
4,dc1eeb61-3ef0-4bfe-b63b-8d52ec7577c5,SNCF:2024-03-18:12254:1187:LongDistanceTrain,TGV,stop_point:SNCF:87581009:LongDistanceTrain,Bordeaux Saint-Jean,-0.556697,44.8259,2024-03-18 10:46:00,2024-03-18 12:58:00,0.0,0.0,Régulation du trafic,2024-03-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50624,dcd5b615-5148-4533-a47e-8566ce884c8a,SNCF:2024-03-26:9877:1187:LongDistanceTrain,TGV,stop_point:SNCF:87713040:LongDistanceTrain,Dijon,5.027260,47.3234,2024-03-26 07:24:00,2024-03-26 15:09:00,5.0,5.0,Régulation du trafic,2024-03-26
50625,dcd5b615-5148-4533-a47e-8566ce884c8a,SNCF:2024-03-26:9877:1187:LongDistanceTrain,TGV,stop_point:SNCF:87723197:LongDistanceTrain,Lyon Part Dieu,4.859410,45.7606,2024-03-26 07:24:00,2024-03-26 15:09:00,5.0,5.0,Régulation du trafic,2024-03-26
50626,dcd5b615-5148-4533-a47e-8566ce884c8a,SNCF:2024-03-26:9877:1187:LongDistanceTrain,TGV,stop_point:SNCF:87725689:LongDistanceTrain,Mâcon,4.824890,46.3026,2024-03-26 07:24:00,2024-03-26 15:09:00,5.0,5.0,Régulation du trafic,2024-03-26
50627,dcd5b615-5148-4533-a47e-8566ce884c8a,SNCF:2024-03-26:9877:1187:LongDistanceTrain,TGV,stop_point:SNCF:87751008:LongDistanceTrain,Marseille Saint-Charles,5.380410,43.3027,2024-03-26 07:24:00,2024-03-26 15:09:00,5.0,5.0,Régulation du trafic,2024-03-26


In [5]:
df_vehicle = pd.read_sql('vehicle_journeys_tgv_ter', engine)
df_vehicle['vehicle_id'] = df_vehicle['vehicle_id'].apply(lambda x: ':'.join(x.split(':')[1:]))
df_vehicle

,vehicle_id,route_id,time_begin,time_end,train_type,id_disruption,data_date
0,SNCF:2024-03-18:117752:1187:LongDistanceTrain,route:SNCF:FR:Line::B10C45A0-C32C-4232-85F2-4B...,05:00:00,07:19:00,Train grande vitesse,0f4e5f2c-a148-438f-b7cb-71b80199e8d4,2024-03-18
1,SNCF:2024-03-18:12244:1187:LongDistanceTrain,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,300d66cc-2614-4087-89d7-89abe997e2cf,2024-03-18
2,SNCF:2024-03-18:12251:1187:LongDistanceTrain,route:SNCF:CSR:421400,07:04:00,09:14:00,Train grande vitesse,None,2024-03-18
3,SNCF:2024-03-18:12254:1187:LongDistanceTrain,route:SNCF:CSR:421400,10:46:00,12:53:00,Train grande vitesse,dc1eeb61-3ef0-4bfe-b63b-8d52ec7577c5,2024-03-18
4,SNCF:2024-03-18:12255:1187:LongDistanceTrain,route:SNCF:CSR:421400,11:04:00,13:14:00,Train grande vitesse,None,2024-03-18
...,...,...,...,...,...,...,...
73070,SNCF:2024-03-26:9890:1187:LongDistanceTrain,route:SNCF:CSR:005400,05:59:00,09:43:00,Train grande vitesse,None,2024-03-26
73071,SNCF:2024-03-26:9894:1187:LongDistanceTrain,route:SNCF:CSR:005400,17:01:00,21:01:00,Train grande vitesse,None,2024-03-26
73072,SNCF:2024-03-26:9896:1187:LongDistanceTrain,route:SNCF:CSR:071100,15:56:00,23:49:00,Train grande vitesse,None,2024-03-26
73073,SNCF:2024-03-26:9898:1187:LongDistanceTrain,route:SNCF:CSR:071200,06:31:00,14:15:00,Train grande vitesse,None,2024-03-26


In [6]:
df_lines = pd.read_sql_table('pt_lines_tgv_ter', engine)
df_lines

,route_id,route_name,train_type,network_name,opening_time,closing_time
0,route:SNCF:0018-80100537-87271007,Dortmund Hbf - Paris Nord,Train grande vitesse,Eurostar,05:18:00,12:21:00
1,route:SNCF:0018-80101840-87271007,Essen Hbf - Paris Nord,Train grande vitesse,Eurostar,07:35:00,22:14:00
2,route:SNCF:0018-80154583-87271007,Köln - Hauptbahnhof - Paris Nord,Train grande vitesse,Eurostar,05:21:00,23:14:00
3,route:SNCF:0018-84000588-87111849,Amsterdam Centraal - Marne-la-Vallée - Chessy,Train grande vitesse,Eurostar,06:47:00,11:05:00
4,route:SNCF:0018-84000588-87271007,Amsterdam Centraal - Paris Nord,Train grande vitesse,Eurostar,06:11:00,23:43:00
...,...,...,...,...,...,...
363,route:stop_area:SNCF:87113001_stop_area:SNCF:8...,Paris Est - Strasbourg,Train grande vitesse,additional service,None,None
364,route:stop_area:SNCF:87394007_stop_area:SNCF:8...,Chartres - Versailles Chantiers,Train grande vitesse,additional service,None,None
365,route:stop_area:SNCF:87411017_stop_area:SNCF:8...,Rouen Rive Droite - Paris Saint-Lazare,Train grande vitesse,additional service,None,None
366,route:stop_area:SNCF:87411017_stop_area:SNCF:8...,Rouen Rive Droite - Le Havre,Train grande vitesse,additional service,None,None


# 

# 

# PREPARATION DES DONNEES 

## Merge des tables 

Probleme que je ne comprend pas encore : il y a beaucoup plus de perturbation dans df_vehicle que dans df_disruption

In [7]:
df_merge_inter = pd.merge(df_disruption.drop(['data_date','train_type'],axis=1), df_vehicle, on=['vehicle_id','id_disruption'], how='left')
df_merge = pd.merge(df_merge_inter, df_lines.drop('train_type', axis=1), on='route_id', how='left')

In [8]:
df_merge_clean = df_merge.dropna(subset=['arrival_delay', 'departure_delay'])
df_merge_clean

,id_disruption,vehicle_id,id_stop,name_stop,lon,lat,disruption_start,disruption_end,arrival_delay,departure_delay,cause_delay,route_id,time_begin,time_end,train_type,data_date,route_name,network_name,opening_time,closing_time
0,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,stop_point:SNCF:87391003:LongDistanceTrain,Paris - Montparnasse - Hall 1 & 2,2.320510,48.8412,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,Défaut d'alimentation électrique,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,2024-03-18,Paris - Montparnasse - Hall 1 & 2 - Bordeaux S...,TGV INOUI,05:17:00,00:46:00
1,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,stop_point:SNCF:87393702:LongDistanceTrain,Massy TGV,2.261250,48.7258,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,Défaut d'alimentation électrique,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,2024-03-18,Paris - Montparnasse - Hall 1 & 2 - Bordeaux S...,TGV INOUI,05:17:00,00:46:00
2,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,stop_point:SNCF:87581009:LongDistanceTrain,Bordeaux Saint-Jean,-0.556697,44.8259,2024-03-18 18:46:00,2024-03-18 21:49:00,0.0,0.0,Défaut d'alimentation électrique,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,2024-03-18,Paris - Montparnasse - Hall 1 & 2 - Bordeaux S...,TGV INOUI,05:17:00,00:46:00
3,dc1eeb61-3ef0-4bfe-b63b-8d52ec7577c5,SNCF:2024-03-18:12254:1187:LongDistanceTrain,stop_point:SNCF:87391003:LongDistanceTrain,Paris - Montparnasse - Hall 1 & 2,2.320510,48.8412,2024-03-18 10:46:00,2024-03-18 12:58:00,5.0,5.0,Régulation du trafic,route:SNCF:CSR:421400,10:46:00,12:53:00,Train grande vitesse,2024-03-18,Paris - Montparnasse - Hall 1 & 2 - Bordeaux S...,TGV INOUI,05:17:00,00:46:00
4,dc1eeb61-3ef0-4bfe-b63b-8d52ec7577c5,SNCF:2024-03-18:12254:1187:LongDistanceTrain,stop_point:SNCF:87581009:LongDistanceTrain,Bordeaux Saint-Jean,-0.556697,44.8259,2024-03-18 10:46:00,2024-03-18 12:58:00,0.0,0.0,Régulation du trafic,route:SNCF:CSR:421400,10:46:00,12:53:00,Train grande vitesse,2024-03-18,Paris - Montparnasse - Hall 1 & 2 - Bordeaux S...,TGV INOUI,05:17:00,00:46:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50624,dcd5b615-5148-4533-a47e-8566ce884c8a,SNCF:2024-03-26:9877:1187:LongDistanceTrain,stop_point:SNCF:87713040:LongDistanceTrain,Dijon,5.027260,47.3234,2024-03-26 07:24:00,2024-03-26 15:09:00,5.0,5.0,Régulation du trafic,route:SNCF:CSR:071100,07:24:00,15:04:00,Train grande vitesse,2024-03-26,Nancy - Nice,TGV INOUI,05:42:00,23:49:00
50625,dcd5b615-5148-4533-a47e-8566ce884c8a,SNCF:2024-03-26:9877:1187:LongDistanceTrain,stop_point:SNCF:87723197:LongDistanceTrain,Lyon Part Dieu,4.859410,45.7606,2024-03-26 07:24:00,2024-03-26 15:09:00,5.0,5.0,Régulation du trafic,route:SNCF:CSR:071100,07:24:00,15:04:00,Train grande vitesse,2024-03-26,Nancy - Nice,TGV INOUI,05:42:00,23:49:00
50626,dcd5b615-5148-4533-a47e-8566ce884c8a,SNCF:2024-03-26:9877:1187:LongDistanceTrain,stop_point:SNCF:87725689:LongDistanceTrain,Mâcon,4.824890,46.3026,2024-03-26 07:24:00,2024-03-26 15:09:00,5.0,5.0,Régulation du trafic,route:SNCF:CSR:071100,07:24:00,15:04:00,Train grande vitesse,2024-03-26,Nancy - Nice,TGV INOUI,05:42:00,23:49:00
50627,dcd5b615-5148-4533-a47e-8566ce884c8a,SNCF:2024-03-26:9877:1187:LongDistanceTrain,stop_point:SNCF:87751008:LongDistanceTrain,Marseille Saint-Charles,5.380410,43.3027,2024-03-26 07:24:00,2024-03-26 15:09:00,5.0,5.0,Régulation du trafic,route:SNCF:CSR:071100,07:24:00,15:04:00,Train grande vitesse,2024-03-26,Nancy - Nice,TGV INOUI,05:42:00,23:49:00


### Rajout des trajets non perturbé

In [9]:
df_vehicle_add = df_vehicle[df_vehicle['id_disruption'].isna()]
df_vehicle_add

,vehicle_id,route_id,time_begin,time_end,train_type,id_disruption,data_date
2,SNCF:2024-03-18:12251:1187:LongDistanceTrain,route:SNCF:CSR:421400,07:04:00,09:14:00,Train grande vitesse,None,2024-03-18
4,SNCF:2024-03-18:12255:1187:LongDistanceTrain,route:SNCF:CSR:421400,11:04:00,13:14:00,Train grande vitesse,None,2024-03-18
6,SNCF:2024-03-18:12259:1187:LongDistanceTrain,route:SNCF:CSR:421400,12:04:00,14:14:00,Train grande vitesse,None,2024-03-18
7,SNCF:2024-03-18:12260:1187:LongDistanceTrain,route:SNCF:CSR:421400,14:46:00,17:03:00,Train grande vitesse,None,2024-03-18
9,SNCF:2024-03-18:12265:1187:LongDistanceTrain,route:SNCF:CSR:421400,13:58:00,16:14:00,Train grande vitesse,None,2024-03-18
...,...,...,...,...,...,...,...
73069,SNCF:2024-03-26:9886:1187:LongDistanceTrain,route:SNCF:CSR:003200,05:59:00,11:01:00,Train grande vitesse,None,2024-03-26
73070,SNCF:2024-03-26:9890:1187:LongDistanceTrain,route:SNCF:CSR:005400,05:59:00,09:43:00,Train grande vitesse,None,2024-03-26
73071,SNCF:2024-03-26:9894:1187:LongDistanceTrain,route:SNCF:CSR:005400,17:01:00,21:01:00,Train grande vitesse,None,2024-03-26
73072,SNCF:2024-03-26:9896:1187:LongDistanceTrain,route:SNCF:CSR:071100,15:56:00,23:49:00,Train grande vitesse,None,2024-03-26


In [10]:
#df_vehicle_add_merge  = pd.merge(df_vehicle_add,df_lines.drop('train_type', axis=1), on='route_id', how='left')
#df_vehicle_add_merge

In [11]:
df_vehicle_add.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63496 entries, 2 to 73073
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   vehicle_id     63496 non-null  object        
 1   route_id       63496 non-null  object        
 2   time_begin     63496 non-null  object        
 3   time_end       63496 non-null  object        
 4   train_type     63496 non-null  object        
 5   id_disruption  0 non-null      object        
 6   data_date      63496 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(6)
memory usage: 3.9+ MB


In [12]:
df_final = pd.concat([df_merge_inter, df_vehicle_add], ignore_index=True)
df_final

,id_disruption,vehicle_id,id_stop,name_stop,lon,lat,disruption_start,disruption_end,arrival_delay,departure_delay,cause_delay,route_id,time_begin,time_end,train_type,data_date
0,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,stop_point:SNCF:87391003:LongDistanceTrain,Paris - Montparnasse - Hall 1 & 2,2.320510,48.8412,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,Défaut d'alimentation électrique,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,2024-03-18
1,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,stop_point:SNCF:87393702:LongDistanceTrain,Massy TGV,2.261250,48.7258,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,Défaut d'alimentation électrique,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,2024-03-18
2,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,stop_point:SNCF:87581009:LongDistanceTrain,Bordeaux Saint-Jean,-0.556697,44.8259,2024-03-18 18:46:00,2024-03-18 21:49:00,0.0,0.0,Défaut d'alimentation électrique,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,2024-03-18
3,dc1eeb61-3ef0-4bfe-b63b-8d52ec7577c5,SNCF:2024-03-18:12254:1187:LongDistanceTrain,stop_point:SNCF:87391003:LongDistanceTrain,Paris - Montparnasse - Hall 1 & 2,2.320510,48.8412,2024-03-18 10:46:00,2024-03-18 12:58:00,5.0,5.0,Régulation du trafic,route:SNCF:CSR:421400,10:46:00,12:53:00,Train grande vitesse,2024-03-18
4,dc1eeb61-3ef0-4bfe-b63b-8d52ec7577c5,SNCF:2024-03-18:12254:1187:LongDistanceTrain,stop_point:SNCF:87581009:LongDistanceTrain,Bordeaux Saint-Jean,-0.556697,44.8259,2024-03-18 10:46:00,2024-03-18 12:58:00,0.0,0.0,Régulation du trafic,route:SNCF:CSR:421400,10:46:00,12:53:00,Train grande vitesse,2024-03-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114120,None,SNCF:2024-03-26:9886:1187:LongDistanceTrain,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,route:SNCF:CSR:003200,05:59:00,11:01:00,Train grande vitesse,2024-03-26
114121,None,SNCF:2024-03-26:9890:1187:LongDistanceTrain,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,route:SNCF:CSR:005400,05:59:00,09:43:00,Train grande vitesse,2024-03-26
114122,None,SNCF:2024-03-26:9894:1187:LongDistanceTrain,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,route:SNCF:CSR:005400,17:01:00,21:01:00,Train grande vitesse,2024-03-26
114123,None,SNCF:2024-03-26:9896:1187:LongDistanceTrain,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,route:SNCF:CSR:071100,15:56:00,23:49:00,Train grande vitesse,2024-03-26


## Création de variables

In [13]:
df_final['disrupted'] = np.where(df_final['id_disruption'].isna(), 0, 1)

In [14]:
df_final['week_day'] = df_final['data_date'].dt.weekday

In [15]:
df_final_v2 = df_final[[#'id_disruption', 
        #'vehicle_id', 
        'id_stop', 
        #'name_stop', 
        #'lon', 
        #'lat',
        'disruption_start',
        'disruption_end', 
        'arrival_delay',
        'departure_delay', 
        'cause_delay',
        'route_id',
        'time_begin',
        'time_end',
        'train_type',
        #'data_date',
        'disrupted',
        'week_day']]

In [16]:
# Création de valeurs binaires pour chaque type de train et jour de la semaine
df_final_v2 = pd.get_dummies(df_final_v2, columns=['train_type','week_day'])

In [17]:
# On créer des tranches d'heures puis indique si le train a circulé dans cette tranche
tranches = list(range(4*60, 25*60, 30))

def train_dans_tranche(depart, arrivee, tranche_debut, tranche_fin):
    return not (arrivee <= tranche_debut or depart >= tranche_fin)

for i in range(len(tranches)-1):
    tranche_nom = f'{tranches[i]//60:02d}h-{tranches[i+1]//60:02d}h'
    df_final_v2[tranche_nom] = df_final_v2.apply(lambda x: train_dans_tranche(x['time_begin'].hour*60 + x['time_begin'].minute, x['time_end'].hour*60 + x['time_end'].minute, tranches[i], tranches[i+1]), axis=1)


In [18]:
# On remplace les trajets et les gares par leur fréquence respective
route_freq = df_final_v2['route_id'].value_counts() / len(df_final_v2)
df_final_v2['route_freq'] = df_final_v2['route_id'].map(route_freq)

stop_freq = df_final_v2['id_stop'].value_counts() / len(df_final_v2[~df_final_v2['id_stop'].isna()])
df_final_v2['stop_freq'] = df_final_v2['id_stop'].map(stop_freq)

In [19]:
# Attribution d'un nombre à chaque cause de perturbation
df_final_v2['cause_delay_label'] = df_final_v2['cause_delay']
df_final_v2['cause_delay'] = pd.factorize(df_final_v2['cause_delay'])[0]

In [20]:
# Regroupement des causes de perturbation selon leurs types 
def regrouper_causes(cause):
    if cause in ["Défaut d'alimentation électrique", 'Défaillance de matériel', 'Panne de signalisation', "Panne d'un passage à niveau", "Panne d'un aiguillage", "Dérangement d'un équipement technique ou informatique", 'Incident technique sur la voie', 'Choc nécessitant une vérification technique sur le train', 'Incident technique sur le car']:
        return 'Problèmes Techniques et Défaillances'
    elif cause in ["Régulation du trafic", "Réutilisation d'un train", 'Travaux sur les voies', 'Difficultés lors de la préparation du train', 'Conditions de départ non réunies', 'Prise en charge de clients en correspondance', 'Saturation des voies en gare', 'Modification de matériel', 'Mise à quai tardive en gare origine', 'Confirmation tardive de la voie', "Erreur d'itinéraire"]:
        return 'Problèmes Logistiques et Opérationnels'
    elif cause in ["Présence d'individus sur les voies", 'Obstacle sur la voie', "Présence d'un bagage abandonné", 'Incident de circulation', 'Accident à un passage à niveau', 'Accident de personne', "Présence d'animaux sur la voie", 'Acte de vandalisme', 'Affluence de voyageurs entraînant un arrêt prolongé', 'Conditions météorologiques', 'Incendie aux abords de la voie', 'Incident sur un train de marchandises', "Ralentissement par manque d'adhérence"]:
        return 'Incidents Externes'
    elif cause in ["Grève SNCF inopinée", "Intervention des forces de l'ordre", 'Assistance à un voyageur malade à bord', "Déclenchement du signal d'alarme", 'Assistance à un voyageur']:
        return 'Interventions et Sécurité'
    else:
        return 'Autres / Non Spécifié'

df_final_v2['cause_delay_grouped'] = df_final_v2['cause_delay_label'].apply(regrouper_causes)

df_final_v2['cause_delay_grouped_label'] = df_final_v2['cause_delay_grouped']
df_final_v2['cause_delay_grouped'] = pd.factorize(df_final_v2['cause_delay_grouped'])[0]

In [21]:
df_final_v2 = df_final_v2.drop(['time_begin', 'time_end','route_id', 'id_stop'], axis=1)

In [22]:
df_final_v2

,disruption_start,disruption_end,arrival_delay,departure_delay,cause_delay,disrupted,train_type_TER / Intercités,train_type_Train grande vitesse,week_day_0,week_day_1,...,22h-22h,22h-23h,23h-23h,23h-24h,24h-24h,route_freq,stop_freq,cause_delay_label,cause_delay_grouped,cause_delay_grouped_label
0,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,0,1,False,True,True,False,...,False,False,False,False,False,0.003470,0.006795,Défaut d'alimentation électrique,0,Problèmes Techniques et Défaillances
1,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,0,1,False,True,True,False,...,False,False,False,False,False,0.003470,0.001304,Défaut d'alimentation électrique,0,Problèmes Techniques et Défaillances
2,2024-03-18 18:46:00,2024-03-18 21:49:00,0.0,0.0,0,1,False,True,True,False,...,False,False,False,False,False,0.003470,0.004089,Défaut d'alimentation électrique,0,Problèmes Techniques et Défaillances
3,2024-03-18 10:46:00,2024-03-18 12:58:00,5.0,5.0,1,1,False,True,True,False,...,False,False,False,False,False,0.003470,0.006795,Régulation du trafic,1,Problèmes Logistiques et Opérationnels
4,2024-03-18 10:46:00,2024-03-18 12:58:00,0.0,0.0,1,1,False,True,True,False,...,False,False,False,False,False,0.003470,0.004089,Régulation du trafic,1,Problèmes Logistiques et Opérationnels
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114120,NaT,NaT,NaN,NaN,-1,0,False,True,False,True,...,False,False,False,False,False,0.000543,NaN,NaN,4,Autres / Non Spécifié
114121,NaT,NaT,NaN,NaN,-1,0,False,True,False,True,...,False,False,False,False,False,0.000464,NaN,NaN,4,Autres / Non Spécifié
114122,NaT,NaT,NaN,NaN,-1,0,False,True,False,True,...,False,False,False,False,False,0.000464,NaN,NaN,4,Autres / Non Spécifié
114123,NaT,NaT,NaN,NaN,-1,0,False,True,False,True,...,True,True,True,True,False,0.001691,NaN,NaN,4,Autres / Non Spécifié


#
#

# PREDIRE SI LE TRAJET VA ETRE PERTURBE OU NON

## 

### RANDOM FOREST (CLASSIFICATION)

In [23]:
df_rf_cla = df_final_v2.drop(['stop_freq',
                'disruption_start',
                'disruption_end',
                'arrival_delay',
                'departure_delay',
                'cause_delay',
                'train_type_Train grande vitesse',
                'train_type_TER / Intercités',
                'cause_delay_label',
                'cause_delay_grouped_label',
                'cause_delay_grouped'],
                             axis=1)

In [24]:
df_rf_cla.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114125 entries, 0 to 114124
Data columns (total 50 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   disrupted   114125 non-null  int32  
 1   week_day_0  114125 non-null  bool   
 2   week_day_1  114125 non-null  bool   
 3   week_day_2  114125 non-null  bool   
 4   week_day_3  114125 non-null  bool   
 5   week_day_4  114125 non-null  bool   
 6   week_day_5  114125 non-null  bool   
 7   week_day_6  114125 non-null  bool   
 8   04h-04h     114125 non-null  bool   
 9   04h-05h     114125 non-null  bool   
 10  05h-05h     114125 non-null  bool   
 11  05h-06h     114125 non-null  bool   
 12  06h-06h     114125 non-null  bool   
 13  06h-07h     114125 non-null  bool   
 14  07h-07h     114125 non-null  bool   
 15  07h-08h     114125 non-null  bool   
 16  08h-08h     114125 non-null  bool   
 17  08h-09h     114125 non-null  bool   
 18  09h-09h     114125 non-null  bool   
 19  09

In [25]:
X_rf_cla = df_rf_cla.drop('disrupted', axis=1)
y_rf_cla = df_rf_cla['disrupted']

X_train_rf_cla, X_test_rf_cla, y_train_rf_cla, y_test_rf_cla = train_test_split(X_rf_cla, y_rf_cla, test_size=0.2, random_state=24)

In [26]:
rf = RandomForestClassifier(class_weight='balanced', random_state=24)
rf.fit(X_train_rf_cla, y_train_rf_cla)

RandomForestClassifier(class_weight='balanced', random_state=24)

In [27]:
# Prédictions
predictions = rf.predict(X_test_rf_cla)

# Évaluation
print(classification_report(y_test_rf_cla, predictions))
print("ROC AUC score:", roc_auc_score(y_test_rf_cla, predictions))

              precision    recall  f1-score   support

           0       0.99      0.89      0.94     12706
           1       0.88      0.99      0.93     10119

    accuracy                           0.93     22825
   macro avg       0.93      0.94      0.93     22825
weighted avg       0.94      0.93      0.93     22825

ROC AUC score: 0.9381512060626196


###  Ajustements des paramètres

In [ ]:
param_grid = {
    'n_estimators': [200,300,400],  
    'max_depth': [None, 10,30,50],  
}

grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='roc_auc')
grid_search.fit(X_train_rf_cla, y_train_rf_cla)

print("Meilleurs paramètres:", grid_search.best_params_)

In [ ]:
rf_best = RandomForestClassifier(**grid_search.best_params_,random_state=10)
rf_best.fit(X_train_rf_cla, y_train_rf_cla)

In [ ]:
# Prédictions
predictions_best = rf_best.predict(X_test_rf_cla)

# Évaluation
print(classification_report(y_test_rf_cla, predictions_best))
print("ROC AUC score:", roc_auc_score(y_test_rf_cla, predictions_best))

In [ ]:
importances = rf_best.feature_importances_

features = X_train_rf_cla.columns
importance_scores = sorted(list(zip(features, importances)), key=lambda x: x[1], reverse=True)

for feature, importance in importance_scores:
    print(f"{feature}: {importance}")

# 

# 

# PREDIRE LE TYPE DE PERTURBATION D'UN TRAJET PERTURBE

## 

### RANDOM FOREST (CLASSIFICATION)

In [28]:
df_rf_cla2 = df_final_v2[df_final_v2['disrupted']==1] 
df_rf_cla2 = df_rf_cla2[df_rf_cla2['cause_delay']>=0] 

df_rf_cla2 = df_rf_cla2.drop(['disruption_start', 
                        'disruption_end', 
                        'arrival_delay',
                        'departure_delay', 
                        'disrupted',
                        'train_type_TER / Intercités',
                        'train_type_Train grande vitesse',
                        'stop_freq',
                        'cause_delay_label',
                        'cause_delay_grouped_label',
                        'cause_delay'],
                        axis=1)

In [29]:
df_rf_cla2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47654 entries, 0 to 50628
Data columns (total 50 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   week_day_0           47654 non-null  bool   
 1   week_day_1           47654 non-null  bool   
 2   week_day_2           47654 non-null  bool   
 3   week_day_3           47654 non-null  bool   
 4   week_day_4           47654 non-null  bool   
 5   week_day_5           47654 non-null  bool   
 6   week_day_6           47654 non-null  bool   
 7   04h-04h              47654 non-null  bool   
 8   04h-05h              47654 non-null  bool   
 9   05h-05h              47654 non-null  bool   
 10  05h-06h              47654 non-null  bool   
 11  06h-06h              47654 non-null  bool   
 12  06h-07h              47654 non-null  bool   
 13  07h-07h              47654 non-null  bool   
 14  07h-08h              47654 non-null  bool   
 15  08h-08h              47654 non-null  bool

In [30]:
X_rf_cla2 = df_rf_cla2.drop('cause_delay_grouped', axis=1)
y_rf_cla2 = df_rf_cla2['cause_delay_grouped']

X_train_rf_cla2, X_test_rf_cla2, y_train_rf_cla2, y_test_rf_cla2 = train_test_split(X_rf_cla2, y_rf_cla2, test_size=0.2, random_state=24)

In [31]:
rf = RandomForestClassifier(random_state=24)
rf.fit(X_train_rf_cla2, y_train_rf_cla2)

RandomForestClassifier(random_state=24)

In [32]:
# Prédiction
predictions = rf.predict(X_test_rf_cla2)

# Évaluation des prédictions
print(classification_report(y_test_rf_cla2, predictions))

              precision    recall  f1-score   support

           0       0.94      0.94      0.94      1283
           1       0.98      0.98      0.98      6528
           2       1.00      0.99      1.00       184
           3       0.93      0.94      0.93      1150
           4       0.95      0.93      0.94       386

    accuracy                           0.97      9531
   macro avg       0.96      0.96      0.96      9531
weighted avg       0.97      0.97      0.97      9531



### Ajustements des paramètres

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='roc_auc')
grid_search.fit(X_train_rf_cla2, y_train_rf_cla2)

print("Meilleurs paramètres:", grid_search.best_params_)

In [ ]:
rf_best = RandomForestClassifier(**grid_search.best_params_,random_state=10)
rf_best.fit(X_train_rf_cla2, y_train_rf_cla2)

In [ ]:
# Prédictions
predictions_best = rf_best.predict(X_test_rf_cla2)

# Évaluation
print(classification_report(y_test_rf_cla2, predictions_best))
print("ROC AUC score:", roc_auc_score(y_test_rf_cla2, predictions_best))

In [ ]:
importances = rf_best.feature_importances_

features = X_train_rf_cla2.columns
importance_scores = sorted(list(zip(features, importances)), key=lambda x: x[1], reverse=True)

for feature, importance in importance_scores:
    print(f"{feature}: {importance}")

# 

# 

# PREDIRE LE RETARD AU DEPART D'UN TRAJET PERTURBE (en minutes)

## 

### RANDOM FOREST (REGRESSION)

In [33]:
df_rf_reg = df_final_v2[df_final_v2['disrupted']==1] 
df_rf_reg = df_rf_reg[df_rf_reg['cause_delay']>=0] 

df_rf_reg = df_rf_reg.drop(['disruption_start', 
                        'disruption_end', 
                        'arrival_delay', 
                        'disrupted',
                        'cause_delay_label',
                        'cause_delay_grouped_label',
                        'cause_delay', 
                        'train_type_Train grande vitesse',
                        'train_type_TER / Intercités'],
                        axis=1)

In [34]:
df_rf_reg = df_rf_reg[~df_rf_reg['departure_delay'].isna()]
df_rf_reg = df_rf_reg[df_rf_reg['departure_delay']>=0]

In [35]:
df_rf_reg = pd.get_dummies(df_rf_reg, columns=['cause_delay_grouped'])

In [36]:
df_rf_reg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45929 entries, 0 to 50628
Data columns (total 56 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   departure_delay        45929 non-null  float64
 1   week_day_0             45929 non-null  bool   
 2   week_day_1             45929 non-null  bool   
 3   week_day_2             45929 non-null  bool   
 4   week_day_3             45929 non-null  bool   
 5   week_day_4             45929 non-null  bool   
 6   week_day_5             45929 non-null  bool   
 7   week_day_6             45929 non-null  bool   
 8   04h-04h                45929 non-null  bool   
 9   04h-05h                45929 non-null  bool   
 10  05h-05h                45929 non-null  bool   
 11  05h-06h                45929 non-null  bool   
 12  06h-06h                45929 non-null  bool   
 13  06h-07h                45929 non-null  bool   
 14  07h-07h                45929 non-null  bool   
 15  07h-08h

In [37]:
X_rf_reg = df_rf_reg.drop('departure_delay', axis=1)
y_rf_reg = df_rf_reg['departure_delay']
X_train_rf_reg, X_test_rf_reg, y_train_rf_reg, y_test_rf_reg = train_test_split(X_rf_reg, y_rf_reg, test_size=0.2, random_state=24)

In [38]:
rf = RandomForestRegressor(random_state=24)
rf.fit(X_train_rf_reg, y_train_rf_reg)

RandomForestRegressor(random_state=24)

In [39]:
# Prédictions de retard en minutes
predictions_reg = rf.predict(X_test_rf_reg)

# Évaluation
print("MSE:", mean_squared_error(y_test_rf_reg, predictions_reg))
print("MAE:", mean_absolute_error(y_test_rf_reg, predictions_reg))
print("R² score:", r2_score(y_test_rf_reg, predictions_reg))

MSE: 90.93135313086111
MAE: 3.0856895282548376
R² score: 0.7480026024042568


### Ajustements des paramètres

In [33]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train_rf_reg, y_train_rf_reg)

print("Meilleurs paramètres:", grid_search.best_params_)

Meilleurs paramètres: {'max_depth': 30, 'min_samples_split': 5, 'n_estimators': 200}


In [34]:
rf_best = RandomForestRegressor(**grid_search.best_params_, random_state=24)
rf_best.fit(X_train_rf_reg, y_train_rf_reg)

RandomForestRegressor(max_depth=30, min_samples_split=5, n_estimators=200,
                      random_state=24)

In [36]:
# Prédictions
predictions_best = rf_best.predict(X_test_rf_reg)

# Évaluation
print("MSE:", mean_squared_error(y_test_rf_reg, predictions_best))
print("MAE:", mean_absolute_error(y_test_rf_reg, predictions_best))
print("R² score:", r2_score(y_test_rf_reg, predictions_best))

MSE: 87.35647406553919
MAE: 3.6273880282359823
R² score: 0.7422953367592013


In [37]:
importances = rf_best.feature_importances_

features = X_train_rf_reg.columns
importance_scores = sorted(list(zip(features, importances)), key=lambda x: x[1], reverse=True)

for feature, importance in importance_scores:
    print(f"{feature}: {importance}")

route_freq: 0.3025380307897859
stop_freq: 0.12209723712302102
week_day_4: 0.08773274873838505
cause_delay_grouped_3: 0.07895787442763694
08h-09h: 0.03360473695938996
week_day_0: 0.025931999119557864
week_day_1: 0.02573592919976729
18h-19h: 0.022620582665787505
13h-14h: 0.0197256265360827
week_day_2: 0.018406003553335232
17h-18h: 0.01756714560213112
19h-20h: 0.01688708517043419
20h-21h: 0.01565397040874246
16h-17h: 0.015648933436073743
11h-12h: 0.015493513266148978
cause_delay_grouped_0: 0.01508834846739115
09h-10h: 0.014996413676223492
12h-13h: 0.014558693455872653
week_day_5: 0.014093995347768704
week_day_3: 0.013561043000752466
06h-07h: 0.01336542349117771
14h-15h: 0.013042372509197184
15h-16h: 0.012743496268776284
22h-23h: 0.011762242571723171
21h-22h: 0.011129102600047873
10h-11h: 0.009072070595763624
week_day_6: 0.008648868108325925
07h-08h: 0.007374940950279444
05h-06h: 0.006545562599583059
cause_delay_grouped_4: 0.0045651994484766385
23h-24h: 0.004315053692050776
cause_delay_gro

## 

### RESAUX DE NEUROS ENTIEREMENT CONNECTE

In [40]:
df_rn = df_rf_reg

#### Normalisation des données


In [41]:
scaler = StandardScaler()

df_rn[['route_freq', 'stop_freq']] = scaler.fit_transform(df_rn[['route_freq', 'stop_freq']])

In [43]:
df_rn.describe().loc[['mean', 'std']]


,departure_delay,route_freq,stop_freq
mean,12.153759,-1.051991e-17,1.386153e-16
std,18.010690,1.000011e+00,1.000011e+00


#### Modèle

In [44]:
X_rn = df_rn.drop('departure_delay', axis=1)
y_rn = df_rn['departure_delay']
X_train_rn, X_test_rn, y_train_rn, y_test_rn = train_test_split(X_rn, y_rn, test_size=0.2, random_state=24)

# 

# 

# PREDIRE LE RETARD A L'ARRIVEE D'UN TRAJET PERTURBE (en minutes)

## 

### RANDOM FOREST (REGRESSION)

In [45]:
df_rf_reg2 = df_final_v2[df_final_v2['disrupted']==1] 
df_rf_reg2 = df_rf_reg2[df_rf_reg2['cause_delay']>=0] 

df_rf_reg2 = df_rf_reg2.drop(['disruption_start', 
                        'disruption_end', 
                        'departure_delay', 
                        'disrupted',
                        'cause_delay_label',
                        'cause_delay_grouped_label',
                        'cause_delay', 
                        'train_type_Train grande vitesse',
                        'train_type_TER / Intercités'],
                        axis=1)

In [46]:
df_rf_reg2 = df_rf_reg2[~df_rf_reg2['arrival_delay'].isna()]
df_rf_reg2 = df_rf_reg2[df_rf_reg2['arrival_delay']>=0]

In [47]:
df_rf_reg2 = pd.get_dummies(df_rf_reg2, columns=['cause_delay_grouped'])

In [48]:
df_rf_reg2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45940 entries, 0 to 50628
Data columns (total 56 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   arrival_delay          45940 non-null  float64
 1   week_day_0             45940 non-null  bool   
 2   week_day_1             45940 non-null  bool   
 3   week_day_2             45940 non-null  bool   
 4   week_day_3             45940 non-null  bool   
 5   week_day_4             45940 non-null  bool   
 6   week_day_5             45940 non-null  bool   
 7   week_day_6             45940 non-null  bool   
 8   04h-04h                45940 non-null  bool   
 9   04h-05h                45940 non-null  bool   
 10  05h-05h                45940 non-null  bool   
 11  05h-06h                45940 non-null  bool   
 12  06h-06h                45940 non-null  bool   
 13  06h-07h                45940 non-null  bool   
 14  07h-07h                45940 non-null  bool   
 15  07h-08h

In [50]:
X_rf_reg2 = df_rf_reg2.drop('arrival_delay', axis=1)
y_rf_reg2 = df_rf_reg2['arrival_delay']
X_train_rf_reg2, X_test_rf_reg2, y_train_rf_reg2, y_test_rf_reg2 = train_test_split(X_rf_reg2, y_rf_reg2, test_size=0.2, random_state=24)

In [51]:
rf = RandomForestRegressor(random_state=24)
rf.fit(X_train_rf_reg2, y_train_rf_reg2)

RandomForestRegressor(random_state=24)

In [52]:
# Prédictions de retard en minutes
predictions_reg2 = rf.predict(X_test_rf_reg2)

# Évaluation
print("MSE:", mean_squared_error(y_test_rf_reg2, predictions_reg2))
print("MAE:", mean_absolute_error(y_test_rf_reg2, predictions_reg2))
print("R² score:", r2_score(y_test_rf_reg2, predictions_reg2))

MSE: 64.82182855112815
MAE: 2.8624506652850603
R² score: 0.7992722037474298


In [53]:
importances = rf.feature_importances_

features = X_train_rf_reg2.columns
importance_scores = sorted(list(zip(features, importances)), key=lambda x: x[1], reverse=True)

for feature, importance in importance_scores:
    print(f"{feature}: {importance}")

route_freq: 0.2594678282405338
stop_freq: 0.12852440614997138
week_day_4: 0.07040268971366614
cause_delay_grouped_3: 0.06810728673971175
week_day_1: 0.032064612271486916
week_day_0: 0.023855769318803635
18h-18h: 0.017731631236494193
week_day_2: 0.01662238429388689
08h-09h: 0.01656918056469325
19h-19h: 0.015175259907452079
cause_delay_grouped_0: 0.015133419043504853
20h-20h: 0.014045752502976292
09h-09h: 0.014040661396059669
18h-19h: 0.013546140115451591
20h-21h: 0.013487798984036792
12h-12h: 0.013416416500962713
week_day_3: 0.012050239456000806
14h-15h: 0.011882866801424167
19h-20h: 0.011745121843270914
10h-10h: 0.011554371648495326
week_day_5: 0.011446006351260639
13h-14h: 0.011247276415984856
16h-17h: 0.010738718062910976
11h-11h: 0.0107386561429546
11h-12h: 0.010696071260259192
17h-18h: 0.009845061547357164
09h-10h: 0.009264596273460662
14h-14h: 0.00915038960104903
21h-21h: 0.009012047492572403
06h-07h: 0.008309542450342445
08h-08h: 0.00827626121052106
21h-22h: 0.00825296867385427
1